## クレジットカードの債務不履行についてのデータセット

データの取得
- Rによる統計的学習入門, 朝倉書店, 2018.<br>
An Introduction to Statistical Learning with Applications in R, Springer, 2017.<br>
https://www.statlearning.com<br>
https://cran.r-project.org/web/packages/ISLR/index.html

データの説明
- default :	クレジットカードの債務不履行
- student : 	学生
- balance : 	クレジットカードの債務残高
- income : 	年収

内容を確認<br>
- "default","student","balance","income"

In [1]:
import pandas as pd

default = pd.read_csv('data/Default.csv')
# 列ごとの欠損値の数
print(default.isna().sum())
# default, student のダミー変数化
df = pd.get_dummies(data=default, drop_first=True)
# データの確認
df.head(2)

default    0
student    0
balance    0
income     0
dtype: int64


,balance,income,default_Yes,student_Yes
0,729.526495,44361.625074,0,0
1,817.180407,12106.134700,0,1


- balance : 	クレジットカードの債務残高
- income : 	年収
- default_Yes :	クレジットカードの債務不履行
- student_Yes : 	学生

In [3]:
# ２．default_Yes を目的変数、他を全て説明変数とした時のAUCはいくつになるか。
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split

def calc_auc(model, X, Y):
    """
    正確度、適合度、再現率（感度）、AUCの表示
    """
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    return auc(fpr, tpr)

X = df.drop(['default_Yes'], axis=1)
Y = df.default_Yes
model = DecisionTreeClassifier(criterion='entropy', max_depth=2)
model.fit(X, Y)
Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
auc(fpr, tpr)

0.9347016924859068

In [18]:
# ３．DataFrame の先頭から default_Yes==0 となるデータを default_Yes==1 と同数だけ取り出し、
# default_Yes==1 の行のデータと組み合わせ、default_Yes==0とdefault_Yes==1のデータ数を同数にした場合のAUCはいくつになるか。

df.query("default_Yes==1").default_Yes.value_counts()

yes_df = df.query("default_Yes==1")

no_df = df.query("default_Yes==0").head(333)
no_df

df_concat = pd.concat([yes_df, no_df])
df_concat

X2 = df_concat.drop(['default_Yes'], axis=1)
Y2 = df_concat.default_Yes
model2 = DecisionTreeClassifier(criterion='entropy', max_depth=2)
model2.fit(X2, Y2)
Y2_proba = model.predict_proba(X2)
fpr2, tpr2, thresholds = roc_curve(Y2, Y2_proba[:, 1])
auc(fpr2, tpr2)

0.9416353290227164

In [19]:
# ４．問3で作成した決定木で、正規化された情報利得の総和が最も大きい説明変数は何か。
pd.DataFrame(model2.feature_importances_, index=X2.columns, columns=['gain']).\
    sort_values('gain', ascending=False)

,gain
balance,1.0
income,0.0
student_Yes,0.0
